Notebook Test

In [1]:
import pandas as pd
from reportlab.pdfgen import canvas
from reportlab.platypus import *
from reportlab.lib.pagesizes import *
from reportlab.lib.styles import *
from reportlab.pdfgen import *
from reportlab.lib import colors
from reportlab.graphics import shapes
from reportlab.graphics.shapes import *

from fpdf import FPDF
from os import path
import time
import numpy as np
from datetime import datetime

Functions to Use Later

In [2]:
def printBanner():
    print()

In [3]:

# Add leading zeros until 7 digits long (IPNs vary in length, need this for merging)
def addLeadingZeros(IPN):
    IPNstr = str(IPN)
    if '.' not in IPNstr:
        if len(IPNstr) == 7:
            return IPNstr
        elif len(IPNstr) == 6:
            tempString = '0'
            tempString += IPNstr
            return tempString
        elif len(IPNstr) == 5:
            tempString = '00'
            tempString += IPNstr
            return tempString
    else:
        IPNsplitted = IPNstr.split('.')
        if (len(IPNsplitted[0])) == 7:
            return IPNstr
        elif (len(IPNsplitted[0])) == 6:
            tempString = '0'
            tempString += IPNstr
            return tempString
        elif (len(IPNsplitted[0])) == 5:
            tempString = '00'
            tempString += IPNstr
            return tempString

In [4]:
# REFERENCE TABLE: DO NOT CHANGE OR DELETE THIS DATA
# Function to apply on Secondary Input
def replaceTableString(tableString):
    x = str(tableString)
    if 'Table31' in x: return 'SX1A L1'
    elif 'Table32' in x: return 'SX1A L2'
    elif 'Table33' in x: return 'SX1B L1'
    elif 'Table34' in x: return 'SX1B L2'
    elif 'Table35' in x: return 'SX2 L1'
    elif 'Table36' in x: return 'SX2 L2'
    elif 'Table51' in x: return 'SX1A L1'
    elif 'Table52' in x: return 'SX1A L2'
    elif 'Table53' in x: return 'SX1B L1'
    elif 'Table54' in x: return 'SX1B L2'
    elif 'Table55' in x: return 'SX2 L1'
    elif 'Table56' in x: return 'SX2 L2'
    elif 'Table65' in x: return 'SX1A L1'
    elif 'Table68' in x: return 'SX1A L2'
    elif 'Table69' in x: return 'SX1B L1'
    elif 'Table70' in x: return 'SX1B L2'
    elif 'Table73' in x: return 'SX2 L1'
    elif 'Table72' in x: return 'SX2 L2'

Reading Inputs

In [5]:
# Reading 2 inputs:
# Continue asking if the first input (Main Input) is not a file path
# Then when entered a real file, do the same thing for Secondary Input
""" print("Files MUST be within the same folder and have extensions '.xlsx'")
mainInput = input("Enter Main Input (MUST be .xlsx): ")
while path.isfile(mainInput) == False:
    mainInput = input("Please re-enter Main Input:")
else:
    secondaryInput = input("Enter Secondary Input (MUST be .xlsx): ") 
    while path.isfile(secondaryInput) == False:
        secondaryInput = input("Please re-enter Secondary Input:") """

# The commented code above is standard, but for the sake of testing I skip asking for user inputs.
mainInput = 'Input 2---Creation_Consolidated_WIP_Job__050723 (1) - Copy.xlsx'
secondaryInput = '1.Query tool.xlsx'

Dealing Main Input

In [6]:
# Reading excel, then adjusting column order based on desired output
dfMainInput = pd.read_excel(mainInput, usecols='B, C, G, H, I', names=['IPN', 'Description', 'Locator', 'OnHand Qty', 'Reqd Qty'], skiprows=7, skipfooter=2)
dfMainInput = dfMainInput[['IPN', 'Description', 'Reqd Qty', 'OnHand Qty', 'Locator']]

# Casting as str, then strip (incase of random spaces or tabs) and add leading zeros
dfMainInput = dfMainInput.astype(str)
dfMainInput['IPN']= dfMainInput['IPN'].str.strip()
dfMainInput['IPN'] = dfMainInput['IPN'].apply(addLeadingZeros)

Reading Job Number - Derived from Main Input file

In [7]:
# Job num appears in C2, so this reads C2 into a string
readJobNumber = pd.read_excel(mainInput, usecols='C', nrows=1)
header = readJobNumber.columns.tolist()
readJobNumber[header[0]] = readJobNumber[header[0]].astype(str)
jobNum = readJobNumber.at[0, header[0]]
print(jobNum)

12419630


Dealing Secondary Input

In [8]:
# Reading excel, then casting to string, then replaces the 'Tor NYAB\' so IPN is proper
dfSecondaryInput = pd.read_excel(secondaryInput, usecols='C, F, G, H', header=None, names=['IPN', 'Table', 'Slot', 'Trk'])
dfSecondaryInput = dfSecondaryInput.astype(str)
dfSecondaryInput['IPN'] = dfSecondaryInput['IPN'].str.replace(r'Tor NYAB\\','', regex=True)

# Applying functions accordingly
dfSecondaryInput['IPN'] = dfSecondaryInput['IPN'].apply(addLeadingZeros)
dfSecondaryInput['Table'] = dfSecondaryInput['Table'].apply(replaceTableString)
#dfSecondaryInput['Table'] = dfSecondaryInput['Table'].str.replace(r'System\\Copy of ','', regex=True)

Merging

In [9]:
# Left merge so that it uses dfMainInput's list of IPNs
dfMerged = dfMainInput.merge(dfSecondaryInput, how="left", on=["IPN"])

# Insert 'Qty Picked' col in the third slot and have it be empty
dfMerged.insert(3, "Qty Picked", "")
# Create 'Comments' col at the end
dfMerged["Comments"] = ""
# Create 'Job' col with repeating jobNum as its value
dfMerged["Job"] = jobNum
dfMerged = dfMerged.replace(np.nan,"", regex=True)
dfMerged = dfMerged.sort_values(['Table', 'Locator'], ascending=[True, True])

In [10]:
""" engine = create_engine('sqlite://', echo=False)
toSQL = dfMerged.to_sql('test', engine)
print(toSQL) """

" engine = create_engine('sqlite://', echo=False)\ntoSQL = dfMerged.to_sql('test', engine)\nprint(toSQL) "

In [11]:
def prepare_df_for_reportlab(df):
    df2 = df.reset_index() # reset the index so row labels show up in the reportlab table
    n = df2.columns.nlevels # number of table header rows to repeat
    if n > 1:
        labels = map(list, zip(*df2.columns.values))
    else:
        labels = [df2.columns[:,].values.astype(str).tolist()]
    values = df2.values.tolist()
    datalist = labels + values
    return datalist, n

REPORTLAB WORKING, BUT STILL NEED TO POLISH

In [12]:
def reprFrame(frame):
    _dict = vars(frame)
    for key in sorted(list(_dict.keys())):
        print(key, ": ", _dict[key])

In [13]:
class NumberedCanvas(canvas.Canvas):
    def __init__(self, *args, **kwargs):
        canvas.Canvas.__init__(self, *args, **kwargs)
        self._saved_page_states = []

    def showPage(self):
        self._saved_page_states.append(dict(self.__dict__))
        self._startPage()

    def save(self):
        """add page info to each page (page x of y)"""
        num_pages = len(self._saved_page_states)
        for state in self._saved_page_states:
            self.__dict__.update(state)
            self.draw_page_number(num_pages)
            canvas.Canvas.showPage(self)
        canvas.Canvas.save(self)

    def draw_page_number(self, page_count):
        #self.drawImage(Rect(50, 50, 300, 100, fillColor=colors.yellow))
        self.setFont("Helvetica-Bold", 10)
        now = datetime.now()
        s = now.strftime('%a %b %d %y')
        self.drawRightString(290*mm, 8*mm,
            "Page %d of %d" % (self._pageNumber, page_count)) # Drawing page number bottom right
        
        self.drawRightString(29*mm, 8*mm, s) # Drawing date at bottom left
        

In [15]:
def test():
    doc = BaseDocTemplate(
        "TestOutput.pdf",
        pagesize=landscape(A4),
        topMargin=0.5*inch,
        bottomMargin=0.5*inch,
        leftMargin=0.1*inch,
        rightMargin=0.1*inch,
    )
    
    elements = []

    dfList = dfMerged.values.tolist()
    #dfList.insert(0,dfMerged.columns)
    data = dfList.copy()
    
    styleHDR = ParagraphStyle(
        name='Normal',
        fontName='Helvetica-Bold',
        fontSize=10,
    )

    styles = getSampleStyleSheet()
    styleN = styles['Normal']
    styleIPN = ParagraphStyle(
        name='Normal',
        fontName='Helvetica-Bold',
        fontSize=10,
    )

    styleDesc = ParagraphStyle(
        name='Normal',
        fontName='Helvetica',
        fontSize=7,
    )


    headerList = (dfMerged.columns.values)
    headerPG = []
    for i in headerList:
        headerPG.append(Paragraph(i, styleHDR))

    data2 = []
    data2.append(headerPG)
    
    for i in range(len(data)):
        tempList = []
        for j in range(11):
            if j == 0: tempList.append(Paragraph(data[i][j], styleIPN))     #IPN
            elif j == 1: tempList.append(Paragraph(data[i][j], styleDesc))  #DESC
            elif j == 2: tempList.append(Paragraph(data[i][j], styleIPN))   #REQD QTY
            elif j == 3: tempList.append(Paragraph(data[i][j], styleN))     #QTY PICKED
            elif j == 4: tempList.append(Paragraph(data[i][j], styleIPN))     #ONHAND QTY
            elif j == 5: tempList.append(Paragraph(data[i][j], styleN))     #LOCATOR
            elif j == 6: tempList.append(Paragraph(data[i][j], styleIPN))     #TABLE
            elif j == 7: tempList.append(Paragraph(data[i][j], styleN))     #SLOT
            elif j == 8: tempList.append(Paragraph(data[i][j], styleN))     #TRK
            elif j == 9: tempList.append(Paragraph(data[i][j], styleN))     #COMMENTS
            elif j == 10: tempList.append(Paragraph(data[i][j], styleN))    #JOB
        data2.append(tempList)



    frame = Frame(doc.leftMargin, doc.bottomMargin, doc.width, doc.height - 0.5 * inch, id='normal')
    reprFrame(frame)


    t1 = Table(data2, 
           colWidths=[0.75*inch,3.4*inch,0.6*inch,1*inch,0.8*inch,1.1*inch,0.8*inch,0.5*inch,0.4*inch,1*inch,0.8*inch], 
           rowHeights=11*mm,
           repeatRows=1,
           )
    
    
    t1.setStyle(TableStyle([
                        ('BACKGROUND', (0,0), (11,0), colors.lightgrey), # Index Column Background
                        ('FONTNAME', (0,0), (11,0), 'Helvetica-Bold'), # Index Column Bold
                        ('FONTSIZE', (0,0), (11,0), 20), # Index Column Bold
                        ('BOX',(0,0),(-1,0),1,colors.black), # BOX
                        ('GRID',(0,0),(-1,-1),1,colors.black), # GRID
                        ('VALIGN',(1,1),(-1,-1),'CENTER'), 
                         
                        ('BOTTOMPADDING',(1,1),(1,-1),5), 
                       ]))

    rowNum = len(dfList)
    for i in range(1, rowNum):
        if i % 2 == 0:
            bc = colors.lightcyan
            ts = TableStyle(
            [('BACKGROUND',(0,i),(-1,i),bc),
            ]
            )
        else:
            bc = colors.white
            ts = TableStyle(
            [('BACKGROUND',(0,i),(-1,i),bc)]
            )

        t1.setStyle(ts)

    t1.setStyle(TableStyle([
                        ('BACKGROUND', (0,0), (11,0), colors.lightgrey), # Index Column Background
                        ('BOX',(0,0),(-1,0),1,colors.black), # BOX
                        ('GRID',(0,0),(-1,-1),1,colors.black), # GRID  
                       ]))


    x = 1
    y = 8   
    width = (8.5-2)*inch # object in points, so we multiply by inch (72 points)
    height = 1*inch # object in points, so we multiply by inch (72 points)
    d = shapes.Drawing(width,height)
    r = shapes.Rect(x, y, width, height, fillColor=None, strokeColor=colors.black)
    # add the rectangle to your Drawing
    d.add(r)

    
    elements.append(t1)
    elements.append(d)
    template = PageTemplate(id='table', frames=frame)
    doc.addPageTemplates([template])
    doc.build(elements, canvasmaker=NumberedCanvas)

test()

_aH :  475.27559055118115
_aW :  815.4897637795276
_atTop :  1
_bottomPadding :  6
_debug :  None
_height :  487.27559055118115
_leftExtraIndent :  0.0
_leftPadding :  6
_oASpace :  1
_prevASpace :  0
_rightExtraIndent :  0.0
_rightPadding :  6
_topPadding :  6
_width :  827.4897637795276
_x :  13.2
_x1 :  7.2
_x2 :  834.6897637795277
_y :  517.2755905511812
_y1 :  36.0
_y1p :  42.0
_y2 :  523.2755905511812
id :  normal
showBoundary :  0
